<a href="https://colab.research.google.com/github/howard-haowen/NLP-demos/blob/main/gensim_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from traitlets.config.manager import BaseJSONConfigManager
from pathlib import Path

In [ ]:
path = Path.home() / ".jupyter" / "nbconfig"
cm = BaseJSONConfigManager(config_dir=str(path))
cm.update(
    "rise",
    {
        "autolaunch": False,
        "enable_chalkboard": True,
        "scroll": True,
        "slideNumber": True,
        "controls": True,
        "progress": True,
        "history": True,
        "center": True,
        "width": "100%",
        "height": "100%",
        "theme": "beige",
        "transition": "concave",
        "start_slideshow_at": "selected"
     }
)

## Dependencies

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install pyldavis
!pip install nltk

In [ ]:
# NLP
import spacy
# corpus
import nltk
# topic modelling
import gensim
from gensim.models import LdaModel, HdpModel
from gensim.corpora import Dictionary
# visualization
import pyLDAvis.gensim_models
# tabularization of data
import pandas as pd
# others
import warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now

## Download the corpus

In [ ]:
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews

In [ ]:
file_ids = movie_reviews.fileids()
len(file_ids)

In [ ]:
movie_reviews.raw(file_ids[0])

In [ ]:
def moviews2df(file_ids):
    sentiments = [1 if id[:3] == 'pos' else 0 for id in file_ids]
    texts = [movie_reviews.raw(id) for id in file_ids]
    df = pd.DataFrame(
        {
            "sentiment": sentiments,
            "text": texts,
        }
    )
    return df

In [ ]:
df = moviews2df(file_ids)
df

## Preprocess data

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def preprocess_texts(texts):
    clean_texts = []
    for text in texts:
        clean_text = [char if char.isascii() else " " for char in text]
        clean_text = "".join(clean_text)
        clean_texts.append(clean_text)
    
    # nlp.pipe() is more efficient than nlp()
    clean_tokens = []
    for doc in nlp.pipe(clean_texts, disable=["ner", "tagger", "parser"]):
        tokens = [tok.lemma_.lower() for tok in doc if (not tok.is_stop) and (not tok.is_punct) and (not tok.like_num) and (not tok.is_space)]
        clean_tokens.append(tokens)

    return clean_tokens

In [ ]:
df['tokens'] = preprocess_texts(df['text'])
df

## Topic modelling

In [ ]:
# corpus = 
# dictionary = 
# model = 

## Visualization

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
# pyLDAvis.gensim_models.prepare(model, corpus, dictionary)